<h1><center> M.Tech (Data Science and Engineering) </center></h1>
<h2><center>Information Retrieval</center></h2>
<h2><center>Assignment 1 - Skip List Implementation</center></h2>

# Question
## A) Explore open-source datasets or document collections where efficient searching is crucial for constructing an inverted index using a skip list. 

There are many datasets and document collections where efficient searching is crucial for constructing an inverted index using a skip list. Few of them are listed below

### DBpedia:

DBpedia is a structured dataset extracted from Wikipedia, providing information in a machine-readable format. Constructing an inverted index can aid in efficient retrieval of facts and relationships.

### The Movie Database (TMDb):

TMDb is a community-driven movie and TV database. Efficient searching can enhance the retrieval of movie-related information for analysis, recommendation systems, and other applications.

### Stack Exchange Datasets:

Stack Exchange provides datasets for various Q&A communities. Constructing an inverted index can enhance the searchability of programming-related questions and answers.

### U.S. Congressional Bills and Resolutions:

The U.S. Congress provides datasets containing bills and resolutions. Constructing an inverted index is essential for efficient searching and analysis of legislative documents.

### Social Media Datasets (e.g., Twitter, Reddit):

Datasets from social media platforms contain a wealth of user-generated content. Efficient searching is crucial for constructing an inverted index to facilitate analysis, sentiment mining, and information retrieval.

## B a) Implement a basic skip list data structure in Python. Explain how a skip list facilitates efficient searching compared to traditional data structures.


A skip list facilitates efficient searching compared to traditional data structures, such as linked lists or binary search trees, by introducing a hierarchical structure with multiple levels of linked lists. The primary advantage of skip lists is that they provide a compromise between the simplicity of linked lists and the efficiency of balanced trees.

Here's how a skip list achieves efficient searching:

**Multiple Levels:**

In a skip list, each element is present in multiple levels of linked lists, with the bottom level containing all elements and higher levels containing a subset of the elements.
Higher levels skip over elements in the lower levels, reducing the number of nodes that need to be traversed during a search.

**Randomization:**

Skip lists use a probabilistic approach to determine the levels at which nodes should appear. This randomization helps balance the tree and prevents worst-case scenarios.
Randomization ensures that the average height of the skip list is logarithmic, leading to an average-case time complexity of O(log n) for search operations.

**Efficient Traversal:**

During a search, the algorithm starts at the top-left corner and moves down and to the right. It only moves down to lower levels when the key in the current node is greater than the target key.
This skipping of nodes at higher levels makes the traversal more efficient, as it eliminates unnecessary comparisons and reduces the number of steps required to find the desired element.

**Balanced Structure:**

Skip lists maintain a balanced structure similar to balanced trees. The randomization of levels ensures that, on average, the number of nodes at each level is limited, preventing skewed structures that could lead to poor performance.

**Dynamic Operations:**

Skip lists support dynamic operations, including insertions and deletions, without requiring a rebalancing process as in some other tree structures.
Dynamic operations can be performed efficiently by adjusting the levels of nodes as needed, maintaining the logarithmic average-case complexity.

**Simple Implementation:**

Compared to balanced trees like AVL or Red-Black trees, skip lists are simpler to implement. They require fewer complex operations during insertion, deletion, and search, making them an attractive choice for scenarios where simplicity is crucial.

**Adaptability:**

Skip lists are adaptable to various data distributions and access patterns. The randomization used in determining levels allows them to perform well across a wide range of scenarios.


#Summary

A skip list facilitates efficient searching by combining a hierarchical structure with randomization. This design reduces the number of comparisons needed during a search, resulting in an average-case time complexity that is logarithmic and comparable to more complex tree structures. The skip list's simplicity and dynamic nature make it a practical choice for scenarios where efficient searching is required without the overhead of complex tree balancing operations.

Skip list is impletemented below from a movie data set from Kaggle

Dataset reference: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata?resource=download

In [1]:
# Basic Skip List Implementation
import random
import pandas as pd


In [2]:
# Node class representing elements in the skip list
class Node:
    def __init__(self, key, level):
        self.key = key
        self.forward = [None] * (level + 1)

In [3]:
# MovieNode class, a subclass of Node, representing movie data nodes
class MovieNode(Node):
    def __init__(self, key, level, movie_data):
        super().__init__(key, level)
        self.movie_data = movie_data

In [4]:
# SkipList class representing the entire skip list
class SkipList:
    def __init__(self, max_level):
        self.max_level = max_level
        self.header = self.create_node("", max_level)  # Header with an empty string as a key
        self.level = 0

    # Method to create a new node
    def create_node(self, key, level, movie_data=None):
        new_node = MovieNode(key, level, movie_data)
        return new_node

    # Method to generate a random level for a new node
    def random_level(self):
        level = 0
        while random.random() < 0.5 and level < self.max_level:
            level += 1
        return level

    # Method to insert a new node into the skip list
    def insert(self, key, movie_data):
        update = [None] * (self.max_level + 1)
        current = self.header

	# Traverse the levels and find the right position for the new node
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current

	# Generate a random level for the new node
        new_level = self.random_level()

	# Update the skip list level if needed
        if new_level > self.level:
            for i in range(self.level + 1, new_level + 1):
                update[i] = self.header
            self.level = new_level


	# Create the new node
        new_node = self.create_node(key, new_level, movie_data)

	# Update pointers to include the new node
        for i in range(new_level + 1):
            new_node.forward[i] = update[i].forward[i]
            update[i].forward[i] = new_node

    # Method to search for nodes containing a given keyword
    def search(self, key):
        current = self.header
        matching_movies = []

        # Determine the number of levels to search, limited to 3 or the current skip list level
        num_levels = min(3, self.level + 1)

	# Iterate through the levels
        for i in range(num_levels):
            while i < len(current.forward) and current.forward[i] is not None:
		# Check if the keyword is present in the current node's key
                if key.lower() in current.forward[i].key.lower():
                    matching_movies.append(current.forward[i].movie_data)
                current = current.forward[i]

	    # Move to the next level if available
            if i + 1 < len(current.forward) and current.forward[i + 1] is not None:
                current = current.forward[i + 1]

        return matching_movies


In [5]:
# Create a SkipList instance with a maximum level of 4
movie_title_skip_list = SkipList(max_level=4)

In [6]:
# Read the dataset and convert it into a list of movie dictionaries
df = pd.read_csv('tmdb_5000_movies.csv')
movies = [{"id": row['id'], "title": row['original_title'], "genres": row['genres'], "overview": row['overview']} for _, row in df.iterrows()]

In [7]:
# Build the skip list for movie titles
for movie in movies:
    movie_title_skip_list.insert(movie["title"], movie)

In [8]:
while True:
    # Example: Search for movies containing the keyword entered by the user
    search_keyword = input("Enter keyword to search for movies or type 'exit' to stop search: ")

    if search_keyword.lower() == 'exit':
        break

    matching_titles = set(movie["title"] for movie in movie_title_skip_list.search(search_keyword))

    # Print the matching movies
    if matching_titles:
        count = len(matching_titles)
        print(f"Total {count} movies are found.")
        print(f"Movies containing '{search_keyword}':")
        count = 0
        for title in matching_titles:
            matching_movies = [movie for movie in movies if movie["title"] == title]
            for matching_movie in matching_movies:
                print(f"{count + 1}. Movie Title: {matching_movie['title']},\nOverview: {matching_movie['overview']}\n")
                count += 1
    else:
        print(f"No movies were found containing '{search_keyword}' .")

Enter keyword to search for movies or type 'exit' to stop search: Action
Total 8 movies are found.
Movies containing 'Action':
1. Movie Title: Fatal Attraction,
Overview: A married man's one night stand comes back to haunt him when that lover begins to stalk him and his family.

2. Movie Title: The Rules of Attraction,
Overview: The incredibly spoiled and overprivileged students of Camden College are a backdrop for an unusual love triangle between a drug dealer, a virgin and a bisexual classmate.

3. Movie Title: Chain Reaction,
Overview: Two researchers in a green alternative energy project are put on the run when they are framed for murder and treason.

4. Movie Title: Laws of Attraction,
Overview: Amidst a sea of litigation, two New York City divorce lawyers find love.

5. Movie Title: Looney Tunes: Back in Action,
Overview: Bugs Bunny and Daffy Duck are up to their feuding ways again. Tired of playing second fiddle to Bugs, Daffy has decided to leave the Studio for good. He is aide

### B b) Modify the skip list implementation to store inverted index entries efficiently. Discuss the advantages and potential challenges of using skip lists for inverted index construction. (4 marks)

Skip lists have both advantages and potential challenges when used for inverted index construction. Here are some key points to consider:

# Advantages:

**Efficient Search Operations:**
Skip lists provide efficient search operations with an average time complexity of O(log n), making them suitable for quickly retrieving information from an inverted index.

**Balanced Structure:**
Skip lists maintain a balanced structure, which ensures that search operations are evenly distributed across levels. This balance contributes to the efficiency of the search process.

**Dynamic Structure:**
Skip lists are dynamic data structures, allowing for easy insertion and deletion of elements. This dynamic nature is particularly beneficial when the inverted index needs to be updated frequently, such as when new documents are added or existing ones are modified.

**Simplicity of Implementation:**
Skip lists are relatively simple to implement compared to other data structures like balanced trees. The simplicity of implementation can be an advantage, especially in scenarios where ease of development is a priority.

**Probabilistic Structure:**
Skip lists use a probabilistic approach to determine the levels of nodes, making them adaptable to various datasets. The randomization involved in determining node levels helps prevent the occurrence of worst-case scenarios.

# Potential Challenges:

**Memory Overhead:**
Skip lists may have higher memory overhead compared to other data structures, such as hash tables. Each node in the skip list requires additional pointers for forward traversal, which can increase the overall memory consumption.

**Complexity of Implementation:**
While skip lists are simpler than some other data structures, they still require careful implementation to ensure correctness. The logic for insertion, deletion, and search operations can be error-prone if not implemented correctly.

**Deterministic Nature of Probabilities:**
The use of randomization in determining node levels is based on probabilities. In certain cases, the deterministic nature of these probabilities might lead to suboptimal performance or even unintended patterns in the skip list structure.

**Limited Performance Guarantees:**
Skip lists offer probabilistic performance guarantees. While they provide average-case time complexity advantages, there is no strict guarantee on the worst-case time complexity. In specific situations, other data structures might provide more predictable performance.

**Limited to Sorted Keys:**
Skip lists are effective when keys are sorted. For inverted index construction, where terms are often sorted, this is an advantage. However, if sorting is not a requirement, other data structures like hash tables might be more appropriate.

#Conclusion
Skip lists offer a good balance between efficiency and simplicity, but application's characteristics and needs should be properly considered before selecting skip lists as the underlying data structure.

Below implementation stores inverted index entries

In [9]:
# Define a basic node structure for the SkipList
class Node:
    def __init__(self, key, level, data=None):
        self.key = key
        self.data = data
        self.forward = [None] * (level + 1)

In [10]:
# Define a specialized node for the Inverted Index SkipList, inheriting from the basic Node
class InvertedIndexNode(Node):
    def __init__(self, key, level, movie_data):
        super().__init__(key, level, movie_data)

In [11]:
# Define the SkipList class with methods for insertion and search
class SkipList:
    def __init__(self, max_level):
        self.max_level = max_level
        self.header = self.create_node("", max_level)  # Header with an empty string as a key
        self.level = 0

    def create_node(self, key, level, movie_data=None):
        new_node = InvertedIndexNode(key, level, movie_data)
        return new_node

    def random_level(self):
        # Randomly determine the level of a new node based on a probability
        # Used for adjusting the level of the SkipList during insertion
        level = 0
        while random.random() < 0.5 and level < self.max_level:
            level += 1
        return level

    def insert(self, key, movie_data):
        # Insert a new node with the provided key and movie_data into the SkipList
        # Adjusts the level of the SkipList as needed and connects the new node
        update = [None] * (self.max_level + 1)
        current = self.header

        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current

        new_level = self.random_level()
        if new_level > self.level:
            for i in range(self.level + 1, new_level + 1):
                update[i] = self.header
            self.level = new_level

        new_node = self.create_node(key, new_level, movie_data)

        for i in range(new_level + 1):
            new_node.forward[i] = update[i].forward[i]
            update[i].forward[i] = new_node

    def search(self, key):
        # Search for nodes containing the specified key in the SkipList
        # Returns a list of matching keys
        current = self.header
        matching_data = set()

        while any(current.forward[i] for i in range(len(current.forward))):
            for i in range(len(current.forward)):
                if current.forward[i] and key.lower() in current.forward[i].key.lower():
                    matching_data.add(current.forward[i].key)
            current = current.forward[0]

        return list(matching_data)

In [12]:
# Create a SkipList instance
inverted_index_skip_list = SkipList(max_level=4)


In [13]:
# Build the Inverted Index SkipList for movie titles
for movie in movies:
    inverted_index_skip_list.insert(movie["title"], movie)

In [14]:
# Continuously prompt the user for keywords and display matching movies
while True:
    # Example: Search for movies containing the keyword entered by the user
    search_keyword = input("Enter the keyword to search for movies or 'exit' to stop search: ")

    if search_keyword.lower() == 'exit':
        break

    matching_titles = inverted_index_skip_list.search(search_keyword)

    # Print the matching movies
    if matching_titles:
        count = len(matching_titles)
        print(f"Total {count} movies found.")
        print(f"Movies containing '{search_keyword}':")
        count = 0
        for title in matching_titles:
            matching_movie = next(movie for movie in movies if movie["title"] == title)
            print(f"{count + 1}. Title: {matching_movie['title']},\nOverview: {matching_movie['overview']}\n")
            count += 1
    else:
        print(f"No movies containing '{search_keyword}' found.")

Enter the keyword to search for movies or 'exit' to stop search: Crime
Total 2 movies found.
Movies containing 'Crime':
1. Title: High Crimes,
Overview: High powered lawyer Claire Kubik finds her world turned upside down when her husband, who she thought was Tom Kubik, is arrested and is revealed to be Ron Chapman. Chapman is on trial for a murder of Latin American villagers while he was in the Marines. Claire soon learns that to navigate the military justice system, she'll need help from the somewhat unconventional Charlie Grimes.

2. Title: El Crimen del Padre Amaro,
Overview: The young Father Amaro is put to the test. He is sent to Mexico to help take care of aging Father Benito when he meets a 16-year-old girl that he begins and affair with. It turns out the girls mother had been having an affair with Father Benito. Father Amaro must soon choose between the holy or the sinful life.

Enter the keyword to search for movies or 'exit' to stop search: exit
